### Codes used to sample a picture.

by ThomasB, in project PyForge

In [53]:
import cv2
import numpy as np

In [ ]:
image_path = "filament.png"
fil_img = cv2.imread(image_path)
if fil_img is None:
    print("Error: Could not load image.")
else:
    print("Image loaded successfully.")
    print(f"Image shape: {fil_img.shape}")

Image loaded successfully.
Image shape: (382, 382, 3)


Validate & calculate step_lengths

In [55]:
img_x = fil_img.shape[0]
img_y = fil_img.shape[1]
step_x = round(img_x / 8)
step_y = round(img_y / 8)
shift_x = round(step_x / 10)
shift_y = round(step_y / 10)
print(f"step_x: {step_x}, step_y: {step_y}, shift_x: {shift_x}, shift_y: {shift_y}")

if fil_img.shape[2] != 3:
    print("Error: Not a RGB image.")

step_x: 48, step_y: 48, shift_x: 5, shift_y: 5


In [56]:
array = []

for i in range(1, 5):
    for j in range(1, 5):
        sum_r = 0
        sum_g = 0
        sum_b = 0

        for k in range(4):
            if k == 0:
                x = (i * 2 - 1) * step_x - shift_x
                y = (j * 2 - 1) * step_y - shift_y
            if k == 1:
                x = (i * 2 - 1) * step_x - shift_x
                y = (j * 2 - 1) * step_y + shift_y
            if k == 2:
                x = (i * 2 - 1) * step_x + shift_x
                y = (j * 2 - 1) * step_y - shift_y
            if k == 3:
                x = (i * 2 - 1) * step_x + shift_x
                y = (j * 2 - 1) * step_y + shift_y

            b, g, r = fil_img[x, y]
            sum_r += int(r)
            sum_g += int(g)
            sum_b += int(b)
        
        average_r = round(sum_r / 4)
        average_g = round(sum_g / 4)
        average_b = round(sum_b / 4)
        array.append([round((1.6 - (i * 4 + j)/10) * 10)/10, np.array([(i * 2 - 1) * step_x, (j * 2 - 1) * step_y]), np.array([average_r, average_g, average_b], dtype=np.uint8)])

print(array)

[[1.1, array([48, 48]), array([ 42, 100, 149], dtype=uint8)], [1.0, array([ 48, 144]), array([248, 222,  75], dtype=uint8)], [0.9, array([ 48, 240]), array([ 42, 100,  25], dtype=uint8)], [0.8, array([ 48, 336]), array([234,  51,  35], dtype=uint8)], [0.7, array([144,  48]), array([234,  51,  35], dtype=uint8)], [0.6, array([144, 144]), array([ 42, 100,  25], dtype=uint8)], [0.5, array([144, 240]), array([248, 222,  75], dtype=uint8)], [0.4, array([144, 336]), array([ 42, 100, 149], dtype=uint8)], [0.3, array([240,  48]), array([248, 222,  75], dtype=uint8)], [0.2, array([240, 144]), array([ 42, 100, 149], dtype=uint8)], [0.1, array([240, 240]), array([234,  51,  35], dtype=uint8)], [0.0, array([240, 336]), array([ 42, 100,  25], dtype=uint8)], [-0.1, array([336,  48]), array([ 42, 100,  25], dtype=uint8)], [-0.2, array([336, 144]), array([234,  51,  35], dtype=uint8)], [-0.3, array([336, 240]), array([ 42, 100, 149], dtype=uint8)], [-0.4, array([336, 336]), array([248, 222,  75], dtyp